# CS 109A - Intro to Data Science: Project (WIP)
## Predicting Loan Outcomes
## Group: Andrew Greene and David Modjeska
### Harvard University, Fall 2016

In [224]:
import itertools as it
import numpy as np
import os.path as op
import pandas as pd
import re
import sklearn.preprocessing as Preprocessing
import datetime

import enchant

from itertools import combinations
from sklearn.cross_validation import train_test_split as sk_split
from sklearn.decomposition import TruncatedSVD as tSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.io import mmwrite

%matplotlib inline
from IPython.display import display, HTML

In [225]:
### specify processed data files to generate - full/partial, partial %, and train/test
### Note: this cell is present in both notebooks

# load and clean full dataset?
load_full = True

# if not loading and cleaning full dataset, what sample percentage?
sample_percent = 10

if load_full:
    pct_str = ""
else: # not load_full
    pct_str = str(sample_percent) + "_pct"
    
### set intermediate file names
dir_str = "./intermediate_files/"

processed_data_file = dir_str + "processed_data_" + pct_str + ".json"

nlp_data_file = dir_str + "nlp_data_" + pct_str + ".json"
term_freqs_file = dir_str + "term_freqs_" + pct_str + ".mtx"
diff_terms_file = dir_str + "diff_terms_" + pct_str + ".json"

## Step 1: Load and Clean Data

### Economic Data

In [226]:
# load CPI data (from https://fred.stlouisfed.org/series/CPIAUCSL/downloaddata)
def load_cpi_data():
    cpi_xls = pd.ExcelFile("datasets/CPIAUCSL.xls")
    cpi_sheet = cpi_xls.sheet_names[0]
    cpi_df = cpi_xls.parse(cpi_sheet, header = None, skiprows = 55)
    cpi_df.columns = ['date', 'cpi']
    
    return cpi_df

In [227]:
# load GDP data (from https://fred.stlouisfed.org/series/GDP/downloaddata)
def load_gdp_data():
    gdp_xls = pd.ExcelFile("datasets/GDP.xls")
    gdp_sheet = gdp_xls.sheet_names[0]
    gdp_df = gdp_xls.parse(gdp_sheet, header = None, skiprows = 20)
    gdp_df.columns = ['date', 'gdp']
    
    return gdp_df

In [228]:
# load unemployment data (from https://fred.stlouisfed.org/series/UNRATE/downloaddata)
def load_unemploy_data():
    unemploy_xls = pd.ExcelFile("datasets/UNRATE.xls")
    unemploy_sheet = unemploy_xls.sheet_names[0]
    unemploy_df = unemploy_xls.parse(unemploy_sheet, header = None, skiprows = 25)
    unemploy_df.columns = ['date', 'unemploy']

    return unemploy_df

In [229]:
### load economic data

econ_filename = "econ_data.json"

if not op.isfile(econ_filename):
    cpi_df = load_cpi_data()
    gdp_df = load_gdp_data()
    unemploy_df = load_unemploy_data()
    
    econ_data_2 = pd.merge(cpi_df, gdp_df, 'inner')
    econ_data_3 = pd.merge(econ_data_2, unemploy_df, 'inner')
    
    econ_data_3.to_json("econ_data.json", date_unit = 's')
    
econ_data_4 = pd.read_json(econ_filename)

In [230]:
# convert UNIX timestamp to calendar quarter
# FIX hack to adjust timezone with timedelta
new_col = econ_data_4["date"].copy()
for index in range(econ_data_4.shape[0]):
        new_col[index] = \
             (datetime.datetime.fromtimestamp(econ_data_4["date"].values[index]) +
            datetime.timedelta(hours = 5)).replace(hour = 0)
econ_data_4['date'] = new_col 

In [231]:
econ_data_4.head()

,cpi,date,gdp,unemploy
0,23.68,1948-01-01 00:00:00,266.2,3.4
1,23.82,1948-04-01 00:00:00,272.9,3.9
10,24.07,1974-10-01 00:00:00,308.5,5.0
100,42.70,1995-04-01 00:00:00,1380.7,4.9
101,43.70,1952-10-01 00:00:00,1417.6,5.0


### LC Data

In [232]:
# helper function to select the columns of interest from the data set
def Select_Data(data):
    
    # list columns to select
    features_to_select = ['id', "loan_status", "annual_inc", "earliest_cr_line", "delinq_2yrs", \
                          "emp_length", "home_ownership", "inq_last_6mths", "loan_amnt", \
                         "purpose", "open_acc", "total_acc", "term", "installment", \
                         "revol_bal", "sub_grade", "issue_d", "int_rate", \
                          'mths_since_last_record', 'emp_title', 'addr_state', \
                          'initial_list_status', 'verification_status', 'recoveries',
                         'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp']
    ratios_to_select = ["dti", "revol_util"]
    text_to_select = ["desc"]

    # concatenate selected columns
    data_select = pd.concat(( \
                            data[features_to_select],
                            data[ratios_to_select], \
                            data[text_to_select]), \
                            axis = 1)

    # synthesize new columns, and drop temporary columns
    monthly_inc = (data["annual_inc"] / 12)
    data_select["ipr"] = data["installment"] / monthly_inc # income to payment ratio
    data_select["rir"] = data["revol_bal"] / monthly_inc # revolving to income ratio
    data_select = data_select.drop("revol_bal", axis = 1)
    
    # rename columns for legibility
    data_select.columns = [
        'id', "loan_status", "annual_income", "earliest_credit", "delinq_2_yrs", \
        "employ_length", "home_owner", "inquiry_6_mos", "loan_amount", \
        "loan_purpose", "open_accounts", "total_accounts", "loan_term", "installment", \
        "loan_subgrade", "issue_date", "interest_rate", "months_since_last_record", \
        "employ_title", "address_state", "initial_list_status", "verif_status", 'recoveries', \
        'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', "dti", "revol_util", "description", \
        "ipr", "rir" \
    ]
    
    return data_select

In [233]:
# helper function to filter the data set down to rows of interest
def Filter_Data(data_select):
    
    # set flags for resolved loans
    status_flags = (data_select["loan_status"] == "Fully Paid") | \
                    (data_select["loan_status"] == "Charged Off")

    # set flags for date range of interest
    earliest_date = pd.to_datetime("2008-01-01")
    issue_dates = pd.to_datetime(data_select["issue_date"])
    date_flags = (issue_dates > earliest_date)
    
    # set flags for 36-month loan terms
    #term_flags = (data_select['loan_term'] == " 36 months")

    # filter rows per flags of interest
    data_filter = \
        data_select.ix[status_flags & date_flags, :].reset_index(drop = True)
    
    return data_filter

In [234]:
# helper function to clean data - recoding, retyping, pruning, and censoring
def Clean_Data(data_filter):
    data_clean = data_filter.copy()

    # recode loan status as boolean: charged off = True
    data_clean["loan_status"] = data_clean["loan_status"] == "Fully Paid"

    # recode loan subgrades from 1 (best) to 35 (worst)
    num_grades = 5
    grade = data_clean["loan_subgrade"].str[0]
    grade = (pd.DataFrame(ord(c) for c in grade) - ord('A')) * num_grades
    sub_grade = data_clean["loan_subgrade"].str[1].astype('int')
    data_clean["loan_subgrade"] =  grade + sub_grade

    # convert earliest credit date to datetime
    data_clean["earliest_credit"] = pd.to_datetime(data_clean["earliest_credit"])

    # prune extra text in loan term 
    data_clean["loan_term"] = data_clean["loan_term"].str.strip()
    data_clean["loan_term"] = data_clean["loan_term"].str.replace(" months", "")


    # prune extra text in employment length, and right-censor
    data_clean["employ_length"] = data_clean["employ_length"].str.replace(" years*", "")
    data_clean["employ_length"] = data_clean["employ_length"].str.replace("10\+", "10")
    data_clean["employ_length"] = data_clean["employ_length"].str.replace("< 1", "0")
    
    # right-censor delinquencies and inquiries
    data_clean["delinq_2_yrs"] = np.clip(data_clean["delinq_2_yrs"], 0, 2)
    data_clean["inquiry_6_mos"] = np.clip(data_clean["inquiry_6_mos"], 0, 3)
        
    return data_clean

In [235]:
# helper function to clean data - filtering nuisance NaNs (not structural NaNs)
def Clean_Data2(data_clean):
    n, p = data_clean.shape
    
    # count nulls by column
    col_nan_pct = data_clean.isnull().sum() / n
    
    # flag columns that have some nuisance nulls
    cols_with_nans = (col_nan_pct > 0.0) & (col_nan_pct < 0.01)
    
    # flag rows that have some nuisance nulls in the flagged columns
    rows_without_nans_flags = data_clean.ix[:, cols_with_nans].notnull()
    
    # index the flagged rows that contain some nuisance nulls
    rows_without_nans_indexes = np.where(rows_without_nans_flags)
    
    # filter the data set to rows that contain no nuisance nulls
    data_clean2 = data_clean.ix[rows_without_nans_indexes[0], :]
    
    return data_clean2

In [236]:
# helper function to pre-process each data subset to get around memory limits
def Prep_Data_Part(index, num_parts, file_prefix, data_all):
    filename = file_prefix + str(index) + ".csv"
    
    # pre-process new data part if the file doesn't already exist
    if not op.isfile(filename):
        n, p = data_all.shape
        
        # pre-process the row range for this data part, avoiding empty data subsets
        start_row = index * (n / num_parts)
        data_part = data_all.ix[range(start_row, start_row + (n / num_parts)), :]
        if data_part.shape[0] > 0:
            data_select = Select_Data(data_part)
            data_filter = Filter_Data(data_select)
            if data_filter.shape[0] > 0:
                data_clean = Clean_Data(data_filter)
                data_clean2 = Clean_Data2(data_clean)
                data_clean2.to_csv(filename, index = False)

In [237]:
# Lending Club (LC) data (from https://www.kaggle.com/wendykan/lending-club-loan-data)
# helper function to pre-process full data set and save new file, 
# or to read pre-processed file if it already exists
# Note: this non-shared intermediate file is not split into train/test
def Preprocess_Full_Dataset():
    file_prefix = "./data_parts/loan_clean_part"
    full_clean_data_file = "loan_clean_data.csv"
    num_parts = 30

    # pre-process data set and save result as new file
    if not op.isfile(full_clean_data_file):

        # pre-process and save part files
        data_raw = pd.read_csv("loan.csv")
        for part in range(num_parts):
            Prep_Data_Part(part, num_parts, file_prefix, data_raw)       

        # read and concatenate part files
        data = pd.DataFrame({})
        for part in range(num_parts):
            file_part = file_prefix + str(part) + ".csv"
            if op.isfile(file_part):
                data_part = pd.read_csv(file_part)
                data = pd.concat((data, data_part), axis = 0)

        # save full file
        data = data.reset_index(drop = True)
        data.to_csv(full_clean_data_file, index = False)

    # read pre-processed full data file
    data = pd.read_csv(full_clean_data_file)
        
    return data

In [238]:
# helper function to pre-process sampled data set and save new file, 
# or to read pre-processed file if it already exists
# Note: this non-shared intermediate file is not split into train/test
def Preprocess_Sample_Dataset():
    sample_clean_data_file = "loan_clean_data_" + str(sample_percent) + "_pct.csv"

    # pre-process sample data set and save result as new file
    if not op.isfile(sample_clean_data_file):
        data_raw = pd.read_csv("loan.csv")
        data_sample, data_other = sk_split(data_raw, train_size = sample_percent / 100.0)
        data_select = Select_Data(data_sample)
        data_filter = Filter_Data(data_select)
        data_clean = Clean_Data(data_filter)
        data_clean2 = Clean_Data2(data_clean)
        data_clean2.to_csv(sample_clean_data_file, index = False)

    # read pre-processed sample data file
    data_clean2 = pd.read_csv(sample_clean_data_file)
        
    return data_clean2

In [239]:
# create or load appropriate version of data set for analysis

if load_full:
    data = Preprocess_Full_Dataset()
    
else:
    data = Preprocess_Sample_Dataset()   

In [240]:
# set boolean and string column data types
data["loan_status"] = data["loan_status"].astype(bool)
data["description"] = data["description"].astype('str')
data["issue_date"] = data["issue_date"].astype('str') # for later conversion to datetime
data["employ_title"] = data["employ_title"].astype('str')
data["address_state"] = data["address_state"].astype('str')

nan_flags = data["description"].str.match("nan")
data.ix[nan_flags, "description"] = None

# replace all numbers with a token
data["description"] = data["description"].str.replace("[0-9]+", "_number_")

In [241]:
# summarize nulls/NaNs in data columns
# FIX - print only cols with nulls
print
print "Count of nulls in data set by column:\n"
print data.isnull().sum()
print


Count of nulls in data set by column:

id                               0
loan_status                      0
annual_income                    0
earliest_credit                  0
delinq_2_yrs                     0
employ_length                    0
home_owner                       0
inquiry_6_mos                    0
loan_amount                      0
loan_purpose                     0
open_accounts                    0
total_accounts                   0
loan_term                        0
installment                      0
loan_subgrade                    0
issue_date                       0
interest_rate                    0
months_since_last_record    217958
employ_title                     0
address_state                    0
initial_list_status              0
verif_status                     0
recoveries                       0
total_rec_int                    0
total_rec_late_fee               0
total_rec_prncp                  0
dti                              0
revol_util     

In [242]:
# create new column for first day of quarter that contains issue date
new_col = data["issue_date"].copy()
new_col.name = 'issue_quarter'
for index in range(data.shape[0]):
    new_col.iloc[index] = datetime.datetime.strptime(data["issue_date"].values[index], 
                                                "%b-%Y").replace(day = 1)
    quarter = (new_col.iloc[index].month - 1) // 3
    new_col.iloc[index] = new_col.iloc[index].replace(month = (3 * quarter) + 1)
data = pd.concat((data, new_col), axis = 1).reset_index(drop = True)

In [243]:
# join LC and economic data
print data.shape
data = data.merge(right = econ_data_4, how = 'inner', 
                  left_on = "issue_quarter", right_on = "date", 
                  left_index = True).reset_index(drop = True)
data.drop('issue_quarter', axis = 1, inplace = True)
data.drop('date', axis = 1, inplace = True)

(248445, 32)


In [244]:
# # normalize float columns
# float_cols = ['dti', 'revol_util', 'ipr', 'rir', 'cpi', 'gdp', 'unemploy']
# data[float_cols] = data[float_cols].astype(float)
# data[float_cols] = Preprocessing.normalize(data[float_cols]).astype(float)

In [245]:
data.head()

,id,loan_status,annual_income,earliest_credit,delinq_2_yrs,employ_length,home_owner,inquiry_6_mos,loan_amount,loan_purpose,...,total_rec_late_fee,total_rec_prncp,dti,revol_util,description,ipr,rir,cpi,gdp,unemploy
0,1077501.0,True,24000.0,1985-01-01,0.0,10,RENT,1.0,5000.0,credit_card,...,0.00,5000.00,27.65,83.7,Borrower added on _number_/_number_/_number_...,0.081435,6.824000,136.2,6218.4,6.8
1,1077430.0,False,30000.0,1999-04-01,0.0,0,RENT,3.0,2500.0,car,...,0.00,456.46,1.00,9.4,Borrower added on _number_/_number_/_number_...,0.023932,0.674800,136.2,6218.4,6.8
2,1077175.0,True,12252.0,2001-11-01,0.0,10,RENT,2.0,2400.0,small_business,...,0.00,2400.00,8.72,98.5,None,0.082595,2.895201,136.2,6218.4,6.8
3,1076863.0,True,49200.0,1996-02-01,0.0,10,RENT,1.0,10000.0,other,...,16.97,10000.00,20.00,21.0,Borrower added on _number_/_number_/_number_...,0.082759,1.365366,136.2,6218.4,6.8
4,1075269.0,True,36000.0,2004-11-01,0.0,3,RENT,3.0,5000.0,wedding,...,0.00,5000.00,11.20,28.3,None,0.052153,2.654333,136.2,6218.4,6.8


In [246]:
data.describe()

,id,annual_income,delinq_2_yrs,inquiry_6_mos,loan_amount,open_accounts,total_accounts,loan_term,installment,loan_subgrade,...,total_rec_int,total_rec_late_fee,total_rec_prncp,dti,revol_util,ipr,rir,cpi,gdp,unemploy
count,2.484450e+05,2.484450e+05,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,...,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000,248445.000000
mean,1.154117e+07,7.241011e+04,0.210143,0.821739,13552.297088,10.930097,25.014957,41.319036,418.061126,11.171297,...,1973.724043,0.720695,11464.281530,16.538901,54.446598,0.078185,2.671392,142.752200,6838.194448,6.607163
std,1.358097e+07,5.765466e+04,0.514727,0.968979,8109.436450,4.870852,11.721059,9.968206,244.505571,6.761935,...,2072.147405,5.451706,8313.225953,7.767674,24.749595,0.040749,2.262012,6.774524,530.949028,0.675307
min,5.473400e+04,3.000000e+03,0.000000,0.000000,500.000000,1.000000,2.000000,36.000000,16.080000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000289,0.000000,113.800000,4900.500000,5.200000
25%,1.452197e+06,4.500000e+04,0.000000,0.000000,7300.000000,7.000000,16.000000,36.000000,240.020000,6.000000,...,651.770000,0.000000,5000.000000,10.760000,36.500000,0.046745,1.201875,139.400000,6492.300000,6.100000
50%,6.180235e+06,6.200000e+04,0.000000,1.000000,12000.000000,10.000000,23.000000,36.000000,365.230000,11.000000,...,1335.360000,0.000000,10000.000000,16.200000,56.000000,0.072771,2.195448,144.500000,6904.200000,6.800000
75%,1.559154e+07,8.700000e+04,0.000000,1.000000,18225.000000,14.000000,32.000000,36.000000,547.160000,16.000000,...,2515.490000,0.000000,16000.000000,21.980000,74.000000,0.104912,3.571549,147.200000,7269.800000,7.100000
max,6.860466e+07,8.706582e+06,2.000000,3.000000,35000.000000,76.000000,150.000000,60.000000,1424.570000,34.000000,...,22777.580000,358.680000,35000.030000,39.990000,892.300000,0.320262,122.614366,154.700000,7893.100000,7.700000


In [247]:
# calculate description lengths in characters
description_flags = data["description"].notnull()
descriptions = data.ix[description_flags, "description"]
description_lengths = descriptions.str.len()
data['desc_len'] = description_lengths

In [248]:
### write data frame to intermediate file 

if not op.isfile(processed_data_file):
    data.to_json(processed_data_file, date_unit = 's')

### Text Analysis

In [249]:
# extract and pre-process loan description and loan_status for NLP
data_nlp = data.loc[description_flags, :].copy()
data_nlp["description"] = data_nlp["description"].str.replace("Borrower.* > ", "")

In [250]:
### set up stemming

from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer(language = 'english', ignore_stopwords = True)
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

def take(n, seq):
    seq = iter(seq)
    result = []
    try:
        for i in range(n):
            result.append(seq.next())
    except StopIteration:
        pass
    
    return result

In [251]:
# stem words in Description field
for index in range(data_nlp.shape[0]):
    data_nlp['description'].values[index] = \
        " ".join(take(1000, stemmed_words(data_nlp['description'].values[index])))

In [252]:
# create n-grams from loan description
vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1, 1))
desc_matrix = vectorizer.fit_transform(data_nlp['description'].values)
n, p = desc_matrix.shape
print desc_matrix.shape

if not op.isfile(term_freqs_file):
    mmwrite(term_freqs_file, desc_matrix)

(87830, 16756)


In [253]:
# apply SVD to document-term matrix
tsvd = tSVD(n_components = 100)
desc_matrix_reduce = tsvd.fit_transform(desc_matrix)

In [254]:
data_nlp['desc_matrix_reduce'] = desc_matrix_reduce[:, 0]

In [255]:
# print descriptive information about n-grams
feature_names = np.array(vectorizer.get_feature_names()).reshape(-1, 1)
print "Number of descriptions and terms:", n, p
print
print "Sample terms:", 
pd.DataFrame(feature_names[:10, 0])

Number of descriptions and terms: 87830 16756

Sample terms:

,0
0,__________________
1,___________________________________
2,_all
3,_far_
4,_number_
5,_number________________
6,_number_a
7,_number_acr
8,_number_am
9,_number_amp


In [256]:
data_nlp['desc_word_count'] = desc_matrix.sum(axis = 1)

In [257]:
data_nlp['vocab_count'] = (desc_matrix > 0).sum(axis=1)
data_nlp['vocab_count_norm'] = data_nlp['vocab_count'] \
    / data_nlp['desc_len'].astype(float) 

In [258]:
# split term matrix into defaulted vs. fully repaid
mask = data_nlp["loan_status"].values == False
bad_term_matrix = desc_matrix[mask]
good_term_matrix = desc_matrix[~mask]

all_term_dict = zip(vectorizer.get_feature_names(),
    np.asarray(desc_matrix.sum(axis = 0)).ravel())
all_term_dict_df = pd.DataFrame(all_term_dict).sort_values(by = [1], \
                                                                   ascending = False)
bad_term_dict = zip(vectorizer.get_feature_names(),
    np.asarray(bad_term_matrix.sum(axis = 0)).ravel())
bad_term_dict_df = pd.DataFrame(bad_term_dict).sort_values(by = [1], \
                                                                   ascending = False)
good_term_dict = zip(vectorizer.get_feature_names(),
    np.asarray(good_term_matrix.sum(axis = 0)).ravel())
good_term_dict_df = pd.DataFrame(good_term_dict).sort_values(by = [1], \
                                                                 ascending = False)

top_bad_dict_df = bad_term_dict_df.iloc[:125, :]
top_good_dict_df = good_term_dict_df.iloc[:125, :]

bad_only_df = pd.DataFrame(list(set(top_bad_dict_df[0]) - set(top_good_dict_df[0])))
good_only_df = pd.DataFrame(list(set(top_good_dict_df[0]) - set(top_bad_dict_df[0])))

In [259]:
print
print "Most Frequent Terms in Descriptions of All Loans:"
all_term_dict_df.head(20)


Most Frequent Terms in Descriptions of All Loans:


,0,1
1964,br,102896
4,_number_,85387
3658,credit,58118
8778,loan,54044
10839,pay,52824
2365,card,51523
4004,debt,42440
10875,payment,30865
3311,consolid,28889
9631,month,22867


In [260]:
print
print "Most Frequent Terms Only in Descriptions of Defaulted Loans:"
bad_only_df


Most Frequent Terms Only in Descriptions of Defaulted Loans:


,0
0,insur
1,feel
2,hard
3,day
4,know
5,mani
6,instead
7,realli
8,investor
9,easier


In [261]:
print
print "Most Frequent Terms Only in Descriptions of Fully Repaid Loans:"
good_only_df


Most Frequent Terms Only in Descriptions of Fully Repaid Loans:


,0
0,faster
1,_number_k
2,excel
3,tax
4,repay
5,colleg
6,low
7,student
8,respons
9,posit


In [262]:
### count misspellings

d = enchant.Dict("en_US")

num_terms = all_term_dict_df.shape[0]
misspellings = np.zeros(num_terms)
for index in range(num_terms):
    misspellings[index] = not d.check(all_term_dict_df.iloc[index, 0])

desc_matrix_misspell = desc_matrix[:, misspellings > 0]
data_nlp['misspell_count'] = desc_matrix_misspell.sum(axis=1)

In [263]:
data_nlp['misspell_count_norm'] = data_nlp['misspell_count'] \
    / data_nlp['desc_len'].astype(float) 

if not op.isfile(nlp_data_file):
    data_nlp.to_json(nlp_data_file, date_unit = 's')

In [264]:
data_nlp.describe()

,id,annual_income,delinq_2_yrs,inquiry_6_mos,loan_amount,open_accounts,total_accounts,loan_term,installment,loan_subgrade,...,cpi,gdp,unemploy,desc_len,desc_matrix_reduce,desc_word_count,vocab_count,vocab_count_norm,misspell_count,misspell_count_norm
count,8.783000e+04,8.783000e+04,87830.000000,87830.000000,87830.000000,87830.000000,87830.000000,87830.000000,87830.000000,87830.000000,...,87830.000000,87830.000000,87830.000000,87830.000000,8.783000e+04,87830.000000,87830.000000,87830.000000,87830.000000,87830.000000
mean,3.162491e+06,7.105433e+04,0.173210,0.838939,13224.949049,10.518092,24.117466,40.439576,410.568232,10.062530,...,138.283607,6466.389439,6.818623,291.893078,1.935219e+00,17.407515,14.561938,0.055685,10.014414,0.034144
std,3.257433e+06,5.494205e+04,0.466481,0.980654,7805.883409,4.585561,11.337558,9.318851,238.687190,6.546622,...,7.150404,462.920444,0.724832,313.937304,3.126043e+00,20.173491,13.219672,0.018073,12.121368,0.016262
min,5.473400e+04,4.000000e+03,0.000000,0.000000,500.000000,1.000000,2.000000,36.000000,16.080000,1.000000,...,113.800000,4900.500000,5.200000,1.000000,-8.571919e-32,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.921268e+05,4.450000e+04,0.000000,0.000000,7200.000000,7.000000,16.000000,36.000000,232.870000,6.000000,...,136.200000,6218.400000,6.600000,120.000000,7.376587e-01,7.000000,6.000000,0.046875,3.000000,0.022222
50%,1.518502e+06,6.000000e+04,0.000000,1.000000,12000.000000,10.000000,23.000000,36.000000,363.990000,9.000000,...,139.400000,6492.300000,7.000000,206.000000,1.105286e+00,12.000000,11.000000,0.058824,7.000000,0.035088
75%,4.526716e+06,8.500000e+04,0.000000,1.000000,18000.000000,13.000000,31.000000,36.000000,536.810000,14.000000,...,143.800000,6829.600000,7.300000,342.000000,2.020144e+00,22.000000,19.000000,0.066860,13.000000,0.045455
max,1.304815e+07,7.141778e+06,2.000000,3.000000,35000.000000,49.000000,99.000000,60.000000,1408.130000,34.000000,...,151.800000,7604.900000,7.700000,5016.000000,1.041826e+02,399.000000,210.000000,0.500000,238.000000,0.500000


### Create NLP Features

In [265]:
#@@ create TF-IDF term matrices for two loan outcomes separately

# split data
data_nlp_2 = data_nlp[['description', 'loan_status']]
good_flags = data_nlp_2['loan_status'] == True
good_nlp = data_nlp_2.loc[good_flags, :]
bad_nlp = data_nlp_2.loc[~good_flags, :]

In [266]:
# compute matrices
vectorizer_good = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 1), norm = None,
                                 use_idf = True)
vectorizer_bad = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 1), norm = None,
                                use_idf = True)
good_matrix = vectorizer_good.fit_transform(good_nlp['description'].values)
bad_matrix = vectorizer_bad.fit_transform(bad_nlp['description'].values)

In [267]:
# sum term scores
good_term_scores = good_matrix.sum(axis = 0)
bad_term_scores = bad_matrix.sum(axis = 0)

good_term_scores = Preprocessing.normalize(good_term_scores, axis = 1)
bad_term_scores = Preprocessing.normalize(bad_term_scores, axis = 1)

In [268]:
# get feature names
good_terms = vectorizer_good.get_feature_names()
bad_terms = vectorizer_bad.get_feature_names()

In [269]:
### calculate absolute differences between normalized term scores

n_good = len(good_terms)
n_bad = len(bad_terms)

good_diffs = np.zeros(n_good)
bad_diffs = np.zeros(n_bad)

# consider all words in good loans
for good_term in range(n_good):
    try: 
        match_index = bad_terms.index(good_terms[good_term])
        good_diffs[good_term] = np.abs(good_term_scores[0, good_term] - 
                                       bad_term_scores[0, match_index])
    except ValueError:
        good_diffs[good_term] = np.abs(good_term_scores[0, good_term])

# # only consider words uniquely in bad loans, since matches already considered above in loop
for bad_term in range(n_bad):
    try:
        match_index = good_terms.index(bad_terms[bad_term])
        bad_diffs[bad_term] = -1
    except ValueError:
        bad_diffs[bad_term] = np.abs(bad_term_scores[0, bad_term])

In [270]:
### find top different terms

good_diffs_df = pd.DataFrame({'diffs': good_diffs, 'terms': good_terms})
bad_diffs_df = pd.DataFrame({'diffs': bad_diffs, 'terms': bad_terms})

diffs_df = pd.concat((good_diffs_df, bad_diffs_df), axis = 0)
diffs_df_sort = diffs_df.sort_values(by = 'diffs', ascending = False)
diffs_df_sort.head(20)

,diffs,terms
10834,0.024974,rate
1,0.023820,_number_
1962,0.023720,busi
9014,0.020848,need
2974,0.016866,consolid
6310,0.013117,help
1746,0.012803,br
15040,0.012555,year
9842,0.011302,payment
1226,0.010864,balanc


In [271]:
### add term flags to data as new features

num_top_diffs = 30
diff_indexes = np.zeros(num_top_diffs).astype(int)
for diff in range(num_top_diffs):
    match_index = np.where(feature_names == diffs_df_sort.iloc[diff, 1])[0]
    diff_indexes[diff] = match_index
    
count_cols = desc_matrix[:, diff_indexes]
count_col_names = feature_names[diff_indexes, 0].astype(str)
count_cols_df = pd.DataFrame(count_cols.toarray(), columns = count_col_names)
if not op.isfile(diff_terms_file):
    count_cols_df.to_json(diff_terms_file, date_unit = 's')